# 問題
問題36から39までは、Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzをコーパスと見なし、統計的な分析を行う。
<br>1行に1記事の情報がJSON形式で格納される。
<br>各行には記事名が”title”キーに、記事本文が”text”キーの辞書オブジェクトに格納され、そのオブジェクトがJSON形式で書き出される。
<br>ファイル全体はgzipで圧縮される。
<br>まず、第3章の処理内容を参考に、Wikipedia記事からマークアップを除去し、各記事のテキストを抽出せよ。そして、コーパスにおける単語（形態素）の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

In [2]:
import pandas as pd
import gzip
import json
import re
import html

# ----------------------------
# マークアップ除去関数
# ----------------------------
def clean_wiki_markup(text: str) -> str:
    """
    Wikipedia記事テキストからマークアップを除去し、プレーンテキストを返す
    """
    if not isinstance(text, str):
        return ""
    text = html.unescape(text)  # HTMLエスケープ解除

    patterns = [
        # --- 巨大ブロック除去 ---
        (r'(?m)^\s*\{\{\s*基礎情報[\s\S]*?^\s*\}\}\s*$', "", re.MULTILINE),  # 基礎情報
        (r'\{\|[\s\S]*?\|\}', "", 0),  # 表 (table)

        # --- ハットノート ---
        (r'\{\{\s*(?:otheruses|Otheruses[^|}]*)\s*\|[^}]*\}\}', "", re.IGNORECASE),

        # --- カテゴリ ---
        (r'\[\[(?:Category|カテゴリ):[^\]]+\]\]', "", re.IGNORECASE),

        # --- HTMLコメント ---
        (r'<!--[\s\S]*?-->', "", 0),

        # --- 強調記法 ---
        (r"'''''(.*?)'''''", r"\1", 0),
        (r"'''(.*?)'''", r"\1", 0),
        (r"''(.*?)''", r"\1", 0),

        # --- 内部リンク処理 ---
        (r"\[\[(?:ファイル|File|画像):[^|\]]+(?:\|[^|\]]+)*\|([^|\]=][^|\]]*)\]\]", r"\1", 0),
        (r"\[\[(?:ファイル|File|画像):([^\]|]+)\]\]", r"\1", 0),
        (r"\[\[[^|\]]+\|([^\]]+)\]\]", r"\1", 0),
        (r"\[\[([^\]|#]+)(?:#[^\]]+)?\]\]", r"\1", 0),
        (r"\[\[:[a-z0-9\-]+:[^|\]]+\|([^\]]+)\]\]", r"\1", re.IGNORECASE),
        (r"\[\[:[a-z0-9\-]+:([^\]#|]+)(?:#[^\]]+)?\]\]", r"\1", re.IGNORECASE),

        # --- テンプレート処理 ---
        (r"\{\{lang\|[a-z0-9\-]+\|([^}]+)\}\}", r"\1", re.IGNORECASE),
        (r"\{\{lang-[a-z0-9\-]+\|([^}]+)\}\}", r"\1", re.IGNORECASE),
        (r"\{\{(?:ill2?|仮リンク)\|([^|}]+)\|[^}]*\}\}", r"\1", re.IGNORECASE),
        (r"\{\{flagicon\|([^|}]+)\}\}", r"\1", re.IGNORECASE),
        (r"\{\{convert\|([^|}]+)\|[^}]*\}\}", r"\1", re.IGNORECASE),

        # --- 参照タグ ---
        (r"(?m)<ref[^>]*>.*?$", "", 0),
        (r"<ref[^>]*>.*?</ref>", "", re.DOTALL | re.IGNORECASE),
        (r"<ref[^>]*/>", "", re.IGNORECASE),
        (r"<references\s*/?>", "", re.IGNORECASE),

        # --- HTMLタグ ---
        (r"<br\s*/?>", "\n", re.IGNORECASE),
        (r"</?(?:small|sup|sub|span|div|center|nowiki|ref)[^>]*>", "", re.IGNORECASE),
        (r"<[^>]+>", "", 0),

        (r"\{\{[^{}]+\}\}", "", 0),   # 単純なテンプレート除去
        (r"\{\{", "", 0),             # 孤立 {{ を除去
        (r"\}\}", "", 0),             # 孤立 }} を除去
    ]

    # ループで繰り返し削除
    changed = True
    while changed:
        old = text
        for pat, repl, flg in patterns:
            text = re.sub(pat, repl, text, flags=(flg | re.DOTALL))
        changed = (text != old)

    # --- 後処理 ---
    text = text.replace("\\n", "\n")  # リテラル \n を改行へ
    text = re.sub(r'(?m)^\s*={2,}\s*(.*?)\s*={2,}\s*$', r'\n\1\n', text)  # 見出しを整形
    text = re.sub(r"[ \t]+", " ", text)  # 余計な空白削除
    text = re.sub(r"\n{3,}", "\n\n", text)  # 改行を整形

    return text.strip()

# ----------------------------
# データの読み込み
# ----------------------------
df = pd.read_json("Jawiki Country.json.gz", lines=True)

print(f"記事数: {len(df)}")

# 最初の記事を確認
raw_text = df["text"].iloc[0]
clean_text = clean_wiki_markup(raw_text)

print("\n=== 元の冒頭200文字 ===")
print(raw_text[:200])
print("\n=== 除去後の冒頭200文字 ===")
print(clean_text[:200])
# 参考：単なる文字列一致ではなく、開始パターンでチェックするのがポイント

記事数: 248

=== 元の冒頭200文字 ===
{{otheruses|主に現代のエジプト・アラブ共和国|古代|古代エジプト}}
{{基礎情報 国
|略名 =エジプト
|漢字書き=埃及
|日本語国名 =エジプト・アラブ共和国
|公式国名 ={{lang|ar|'''جمهورية مصر العربية'''}}
|国旗画像 =Flag of Egypt.svg
|国章画像 =[[ファイル:Coat_of_arms_of_Egypt.svg|1

=== 除去後の冒頭200文字 ===
エジプト・アラブ共和国（エジプト・アラブきょうわこく、جمهورية مصر العربية）、通称エジプトは、中東（アラブ世界）および北アフリカにある共和国。首都はカイロ。

アフリカ大陸では北東端に位置し、西にリビア、南にスーダン、北東のシナイ半島ではイスラエル、ガザ地区と国境を接する。北は地中海、東は紅海に面している。南北に流れるナイル川の河谷とデルタ地帯（ナイル・デルタ）のほかは、国土の大


In [3]:
from janome.tokenizer import Tokenizer
from collections import Counter

t = Tokenizer()
freq = Counter()

for raw in df["text"]:
    cleaned = clean_wiki_markup(raw)
    for token in t.tokenize(cleaned):
        freq.update([token.base_form])

# 上位20件
for w, c in freq.most_common(20):
    print(w, c)

の 71589
、 68393
  42406
は 42020
に 40076
。 39217
する 38494
が 36283
を 30853
た 28401


 26710
と 21945
て 20756
年 19390
だ 17553
で 17082

 15706
れる 14913
（ 14465
） 14188


In [15]:
import json
import gzip
import re
import MeCab
from collections import Counter


def remove_markup(text):
    # 強調マークアップの除去
    text = re.sub(r"\'{2,5}", "", text)
    # 内部リンクの除去
    text = re.sub(r"\[\[(?:[^|\]]*?\|)??([^|\]]+?)\]\]", r"\1", text)
    # 外部リンクの除去
    text = re.sub(r"\[http://[^\]]+\]", "", text)
    # HTMLタグの除去
    text = re.sub(r"<[^>]+>", "", text)
    # テンプレートの除去
    text = re.sub(r"\{\{.*?\}\}", "", text)
    return text


def analyze_word_frequency():
    # MeCabの初期化
    mecab = MeCab.Tagger("-Owakati")

    # 単語の出現頻度をカウントするためのCounter
    word_counter = Counter()

    # gzipファイルを読み込む
    with gzip.open("Jawiki Country.json.gz", "rt", encoding="utf-8") as f:
        print("行単位で読み込まれている。次のfor文で1行ずつ取り出している。dfよりも軽い。;",f)
        for line in f:
            sentence = json.loads(line)
            text = sentence["text"]

            # マークアップを除去
            text = remove_markup(text)

            # 形態素解析を行い、単語をカウント
            words = mecab.parse(text).strip().split()
            word_counter.update(words)

    # 出現頻度の高い20語を表示
    for word, count in word_counter.most_common(20):
        print(f"{word}: {count}")


analyze_word_frequency()

行単位で読み込まれている。次のfor文で1行ずつ取り出している。dfよりも軽い。; <_io.TextIOWrapper name='Jawiki Country.json.gz' encoding='utf-8'>
の: 90155
、: 86498
=: 66263
に: 62472
。: 52679
は: 51716
|: 50390
が: 43756
を: 38629
て: 36008
た: 35620
と: 34498
で: 34428
し: 29917
年: 28828
.: 21057
（: 19024
）: 18646
・: 18597
/: 13986
